<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_11_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/Gestures.csv'
data = pd.read_csv(url)

In [67]:
# [1]

# select arctan variables
data1 = data.iloc[:, 3:82]

# replace all numbers other than 0 with 1
for i in range(79):
    data1.iloc[:, i] = data1.iloc[:, i].apply(lambda x: 0 if x == 0 else 1)

# count all "1"s
count = data1.sum().sum()

print(f'Answer [1] : {count}')

Answer [1] : 469041


In [68]:
# [2] ANOVA

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import math


# get column name list
cols = data.columns

# create total distand column
data2 = data.assign(ttl_dist=data[cols[82:]].sum(axis=1))

# perform ANOVA test
ols_reg = ols(formula='ttl_dist ~ C(character)', data=data2).fit()
anova_table = anova_lm(ols_reg)

# decision
if anova_table['PR(>F)'].iloc[0] < 0.05:
    reject = 'YES'
else:
    reject = 'NO'

# get F-value
f_val = math.floor(anova_table.F.iloc[0])

print(f'Answer [2] : Dist by Char is SAME, Reject this? {reject}, {f_val}')

Answer [2] : Dist by Char is SAME, Reject this? YES, 814


In [69]:
# [3] PCA, StandardScaler

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# normalization
norm = StandardScaler().fit_transform(data[cols[3:]])

# get principal components
pca = PCA(n_components=15, random_state=1234).fit(norm)

# get percentage of variance explained
explain = pca.explained_variance_ratio_.sum() * 100

# get component
comp = pca.components_[1][2]

print(f'Answer [3] : {explain:.4f}, {comp:.4f}')

Answer [3] : 57.8805, 0.0658


In [70]:
# [4] PCA, KNN

from sklearn.neighbors import KNeighborsClassifier

# prepare training and test set
data4 = pca.transform(norm)     # transform PCA data
data4 = pd.DataFrame(data4)     # convert to DataFrame
data4 = pd.concat([data[['seq', 'character', 'XGRP']], data4], axis=1)

train_X = data4[data4.XGRP == 'TRAIN'].iloc[:, 3:]
train_y = data4[data4.XGRP == 'TRAIN'].iloc[:, 1:2]
test_X = data4[data4.XGRP == 'TEST'].iloc[:, 3:]
test_y = data4[data4.XGRP == 'TEST'].iloc[:, 1:2]

# perform KNN
# reday to estimate new(test) point's class
knn = KNeighborsClassifier(n_neighbors=data4.character.nunique()).fit(
                                               train_X, train_y.to_numpy().T[0])
                                                        # convert to 1-d array
# estimate with test data
pred = knn.predict(test_X)

# calculate accuracy
test_y.reset_index(inplace=True) # reset test_y to concat with pred
test_y = test_y.drop(columns='index')
pred = pd.DataFrame(pred)   # convert pred to DataFrame
accu = pd.concat([test_y, pred], ignore_index=True, axis=1) # concat actual and
                                                            # pred
accu = accu.assign(accuracy=accu.apply(lambda x: 1 if x[0] == x[1] else 0,
                                       axis=1))

result = accu.accuracy.sum() / accu.accuracy.count()

print(f'Answer [4] : {result:.3f}')

Answer [4] : 0.817
